<a href="https://colab.research.google.com/github/SaiRajesh228/DA6401_Assignment2/blob/main/DA6401_Assignment2_PartA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset, RandomSampler
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import wandb

# Mount Google Drive if in Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
    DATA_ROOT = '/content/drive/MyDrive/inaturalist_12K'
except ImportError:
    DATA_ROOT = './data'

# ---------------------------
# Data Manager
# ---------------------------
class ImageDataManager:
    """Handles image dataset preparation: loading, normalization, augmentation"""
    def __init__(self, img_size, data_root, device, standardize=True):
        self.img_size = img_size
        self.data_root = data_root
        self.device = device
        self.standardize = standardize
        self.mean, self.std = None, None

    def _compute_stats(self, subset):
        trans = transforms.Compose([transforms.Resize(self.img_size), transforms.ToTensor()])
        dataset = datasets.ImageFolder(os.path.join(self.data_root, subset), trans)
        loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=2)

        mean = torch.zeros(3).to(self.device)
        var = torch.zeros(3).to(self.device)
        total = 0
        for imgs, _ in loader:
            imgs = imgs.to(self.device)
            b = imgs.size(0)
            imgs_flat = imgs.view(b, 3, -1)
            mean += imgs_flat.mean(2).sum(0)
            var  += imgs_flat.var(2).sum(0)
            total += b
        self.mean = (mean/total).cpu()
        self.std  = (torch.sqrt(var/total)).cpu()
        return self.mean, self.std

    def create_loader(self, subset, batch_size=32, augmentations=None):
        base = [transforms.Resize(self.img_size), transforms.ToTensor()]
        if self.standardize:
            if self.mean is None:
                self._compute_stats(subset)
            base.append(transforms.Normalize(self.mean, self.std))

        trans_list = list(base)
        if subset.startswith('train') and augmentations:
            datasets_list = []
            datasets_list.append(datasets.ImageFolder(
                os.path.join(self.data_root, subset), transforms.Compose(trans_list)
            ))
            for aug in augmentations:
                datasets_list.append(datasets.ImageFolder(
                    os.path.join(self.data_root, subset), transforms.Compose(aug + trans_list)
                ))
            final_ds = ConcatDataset(datasets_list)
        else:
            final_ds = datasets.ImageFolder(
                os.path.join(self.data_root, subset), transforms.Compose(trans_list)
            )
        return DataLoader(final_ds, batch_size=batch_size,
                          shuffle=subset.startswith('train'),
                          num_workers=2, pin_memory=True)

# ---------------------------
# CNN Model
# ---------------------------
class CustomCNN(nn.Module):
    """Flexible CNN with conv, pool, batch-norm, dropout and dense layers"""
    def __init__(self, input_size, in_channels, num_classes,
                 conv_layers, dense_units,
                 conv_activation=nn.ReLU, fc_activation=nn.ReLU,
                 use_bn=True, dropout_rate=0.0):
        super().__init__()
        h, w = input_size
        current_c = in_channels
        self.features = nn.Sequential()
        layer_idx = 0
        for cfg in conv_layers:
            if cfg['type']=='conv':
                self.features.add_module(f"conv{layer_idx}",
                                         nn.Conv2d(current_c, cfg['filters'],
                                                   cfg['kernel'], cfg['stride'], cfg['padding']))
                if use_bn:
                    self.features.add_module(f"bn{layer_idx}", nn.BatchNorm2d(cfg['filters']))
                self.features.add_module(f"act{layer_idx}", conv_activation())
                current_c = cfg['filters']
                h = (h - cfg['kernel'] + 2*cfg['padding'])//cfg['stride'] + 1
                w = (w - cfg['kernel'] + 2*cfg['padding'])//cfg['stride'] + 1
            elif cfg['type']=='pool':
                self.features.add_module(f"pool{layer_idx}",
                                         nn.MaxPool2d(cfg['size'], cfg['stride']))
                h = (h - cfg['size'])//cfg['stride'] + 1
                w = (w - cfg['size'])//cfg['stride'] + 1
            layer_idx +=1

        flatten_dim = h * w * current_c
        seq = []
        in_feat = flatten_dim
        for i, u in enumerate(dense_units):
            seq.append(nn.Linear(in_feat, u))
            seq.append(fc_activation())
            if dropout_rate >0:
                seq.append(nn.Dropout(dropout_rate))
            in_feat = u
        seq.append(nn.Linear(in_feat, num_classes))
        self.classifier = nn.Sequential(*seq)
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                m.bias.data.zero_()

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x,1)
        return self.classifier(x)

# ---------------------------
# Experiment Pipeline
# ---------------------------
class DLExperiment:
    """Manages end-to-end training, evaluation, and visualization"""
    def __init__(self, img_size, data_root, device=None, use_wandb=False):
        self.device = device or torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.data_mgr = ImageDataManager(img_size, data_root, self.device)
        self.use_wandb = use_wandb
        self.class_names = None

    def setup_data(self, batch_size, data_aug=None):
        aug_presets = []
        if data_aug:
            aug_list = [ [transforms.RandomHorizontalFlip()],
                         [transforms.ColorJitter(brightness=0.3)] ]
            aug_presets = aug_list[:data_aug]
        self.train_loader = self.data_mgr.create_loader('train', batch_size, augmentations=aug_presets)
        self.val_loader   = self.data_mgr.create_loader('val',   batch_size)
        self.test_loader  = self.data_mgr.create_loader('test',  batch_size)
        ds = (self.train_loader.dataset if not isinstance(self.train_loader.dataset, ConcatDataset)
              else self.train_loader.dataset.datasets[0])
        self.class_names = ds.classes

    def setup_model(self, conv_config, dense_units, num_classes,
                    use_bn=True, dropout_rate=0.0):
        self.model = CustomCNN(
            input_size=self.data_mgr.img_size,
            in_channels=3, num_classes=num_classes,
            conv_layers=conv_config, dense_units=dense_units,
            use_bn=use_bn, dropout_rate=dropout_rate
        ).to(self.device)

    def train(self, epochs, lr, weight_decay):
        optimizer = optim.Adam(self.model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = nn.CrossEntropyLoss()
        best_acc = 0.0
        for epoch in range(1, epochs+1):
            self.model.train()
            total_loss=0; total=0; correct=0
            for imgs, labels in self.train_loader:
                imgs, labels = imgs.to(self.device), labels.to(self.device)
                optimizer.zero_grad()
                out = self.model(imgs)
                loss = criterion(out, labels)
                loss.backward(); optimizer.step()
                total_loss += loss.item()*imgs.size(0)
                _, pred = out.max(1); total+=labels.size(0)
                correct += pred.eq(labels).sum().item()
            train_acc = 100*correct/total
            val_loss, val_acc = self.evaluate(self.val_loader)
            print(f"Epoch {epoch}/{epochs} - Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")
            if val_acc>best_acc:
                best_acc=val_acc
                torch.save(self.model.state_dict(), 'best_model.pth')

    def evaluate(self, loader):
        self.model.eval(); total=0; correct=0; loss=0.0; crit=nn.CrossEntropyLoss()
        with torch.no_grad():
            for imgs, labels in loader:
                imgs, labels = imgs.to(self.device), labels.to(self.device)
                out = self.model(imgs)
                loss += crit(out, labels).item()*imgs.size(0)
                _, pred = out.max(1); total+=labels.size(0)
                correct+=pred.eq(labels).sum().item()
        return loss/total, 100*correct/total

    def visualize(self, num_images=9):
        self.model.eval()
        sampler = RandomSampler(self.test_loader.dataset)
        imgs, labels = next(iter(DataLoader(self.test_loader.dataset, batch_size=num_images, sampler=sampler)))
        with torch.no_grad(): preds = self.model(imgs.to(self.device)).argmax(1)
        plt.figure(figsize=(8,8))
        for i in range(num_images):
            plt.subplot(3,3,i+1)
            im = imgs[i].permute(1,2,0).cpu().numpy()
            plt.imshow(np.clip(im,0,1))
            color = 'green' if preds[i]==labels[i] else 'red'
            plt.title(f"True:{self.class_names[labels[i]]}\nPred:{self.class_names[preds[i]]}", color=color)
            plt.axis('off')
        plt.tight_layout(); plt.show()

# ---------------------------
# Configuration and Execution
# ---------------------------
def run_experiment(config, sweep=False):
    # Initialize wandb to the specific project
    if sweep and wandb.run is None:
        wandb.init(project="DA6401_Assignment2_PartA", config=config)
    elif not sweep:
        wandb.init(project="DA6401_Assignment2_PartA", config=config)

    dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    exp = DLExperiment(img_size=(config['crop_size'],config['crop_size']),
                       data_root=DATA_ROOT, device=dev, use_wandb=True)
    exp.setup_data(batch_size=config['batch_size'], data_aug=config.get('data_aug',0))

    conv_cfg = []
    filters = config['num_filters']
    for i in range(config['conv_layers']):
        conv_cfg.append({'type':'conv','filters':filters,'kernel':config['filter_size'],
                         'stride':1,'padding':config['filter_size']//2})
        conv_cfg.append({'type':'pool','size':2,'stride':2})
        filters *= config['filter_growth_factor']

    exp.setup_model(conv_cfg,
                    dense_units=[config['hidden_units']]*config['dense_layers'],
                    num_classes=len(exp.class_names),
                    use_bn=config['batch_norm'],
                    dropout_rate=config.get('dropout_rate',0.0))

    exp.train(epochs=config['training_epochs'],
              lr=config['learning_rate'],
              weight_decay=config['l2_regularization'])
    exp.visualize()

# Example config dict
sample_config = {
    'conv_layers': 4,
    'num_filters': 32,
    'filter_size': 3,
    'filter_growth_factor': 2,
    'dense_layers': 2,
    'hidden_units': 512,
    'batch_norm': True,
    'dropout_rate': 0.2,
    'data_aug': 2,
    'crop_size': 600,
    'batch_size': 32,
    'learning_rate': 1e-3,
    'l2_regularization': 1e-4,
    'training_epochs': 10
}

if __name__ == '__main__':
    run_experiment(sample_config)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/inaturalist_12K/test'